# Fakes and Dependency Injection

## I created a super cool library to manage your bank account.

In [1]:
class BankAccount:
    def __init__(self, iban):
        """Connect to a bank account."""

    def withdraw_money(self, amount):
        """Return some ``amount`` of money."""

    def get_balance(self):
        """Return how much money you have on your bank account."""

## And you decided to use it.

In [2]:
def buy_something(price):
    bank = BankAccount('DE51-1234')
    cash = bank.withdraw_money(price)

    something = give_money(cash)
    return something

## To be sure your code works properly, you wrote a basic test.

In [3]:
def test_buy_something():
    bank = BankAccount('DE51-1234')
    balance = bank.get_balance()

    buy_something(25)
    assert bank.get_balance() == (balance - 25)

## Unfortunately, every time you run the tests, money disappeared from your bank account.

## That's why you decided to write a fake implementation of my cool library.

In [4]:
class FakeBankAccount:
    def __init__(self, iban):
        self.iban = iban
        self.balance = 0
        self.withdrawals = []

    def withdraw_money(self, amount):
        self.withdrawals.append(amount)

    def get_balance(self):
        return self.balance - sum(self.withdrawals)

## Then, you refactored a bit your test, to be able to inject your new Fake into your code.

In [5]:
def test_buy_something():
    bank = FakeBankAccount('DE51-1234')
    balance = bank.get_balance()
    
    buy_something(25, using=bank)  # Inject "bank" into your code
    assert bank.get_balance() == (balance - 25)

In [6]:
def buy_something(price, using=None):
    bank = using if using else BankAccount('DE51-1234')
    cash = bank.withdraw_money(price)
    something = give_money(cash)
    return something

## Finally, you wrote an interface test, to be sure that your (fake) implementation stays up-to-date with mine (the real one).

In [7]:
class TestBankAccount:
    def test_withdraw_money(self, bank):
        balance = bank.get_balance()
        bank.withdraw_money(100)
        assert bank.get_balance() == (balance - 100)


if __name__ == '__main__':
    interface_tests = TestBankAccount()
    
    for bank in [FakeBankAccount('DE51-1234'), BankAccount('DE51-1234')]:
        interface_tests.test_withdraw_money(bank)